In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from qmplot import manhattanplot
import re
from numpy.linalg import inv
from pathlib import Path 
from scipy.stats import chisquare
import scipy.stats as stats  

In [3]:
df_candidate = pd.read_table("../output_csv/df_candidate.csv", low_memory=False, sep=";")

In [4]:
df_candidate

,chromosome,base_pair_location,effect_allele,other_allele,fi_z,fg_z,hba1c_z,t2d_z,bmi_z,gcn_z
0,1,693731,A,G,0.393443,-1.322034,-0.129630,0.369748,-0.193548,-0.223242
1,1,180518603,T,C,-0.340426,1.219512,1.566667,0.388489,-0.390244,-1.887906
2,1,180518806,A,C,-0.097561,-0.297297,0.444444,-1.069231,0.780488,0.098940
3,1,180519091,T,G,0.666667,-0.086957,-0.705882,-1.675000,-1.875000,-1.523077
4,1,180519109,T,C,-0.877551,-0.046512,-2.000000,0.022222,-1.076923,-0.805471
...,...,...,...,...,...,...,...,...,...,...
44245761,24,1832945,A,G,NaN,NaN,-0.632013,NaN,NaN,NaN
44245762,24,1832475,T,C,NaN,NaN,0.172690,NaN,NaN,NaN
44245763,25,13276,T,C,NaN,NaN,NaN,NaN,-1.918919,NaN
44245764,26,24385419,A,G,NaN,NaN,NaN,NaN,0.289474,NaN


In [5]:
z = df_candidate[['fi_z', 'fg_z', 'hba1c_z', 't2d_z', 'bmi_z', 'gcn_z']].to_numpy()

In [6]:
z

array([[ 0.39344262, -1.3220339 , -0.12962963,  0.3697479 , -0.19354839,
        -0.22324159],
       [-0.34042553,  1.2195122 ,  1.56666667,  0.38848921, -0.3902439 ,
        -1.8879056 ],
       [-0.09756098, -0.2972973 ,  0.44444444, -1.06923077,  0.7804878 ,
         0.09893993],
       ...,
       [        nan,         nan,         nan,         nan, -1.91891892,
                nan],
       [        nan,         nan,         nan,         nan,  0.28947368,
                nan],
       [        nan,         nan,         nan,         nan, -1.125     ,
                nan]])

In [6]:
z.shape

(44245766, 6)

In [7]:
correlation = np.loadtxt('correlation.txt', usecols=range(6))

In [8]:
correlation

array([[ 1.        ,  0.19626832,  0.04210314,  0.06468955, -0.02090571,
         0.01468997],
       [ 0.19626832,  1.        ,  0.12033641,  0.11429107, -0.01053008,
        -0.00171142],
       [ 0.04210314,  0.12033641,  1.        ,  0.11435733,  0.03385854,
         0.0039523 ],
       [ 0.06468955,  0.11429107,  0.11435733,  1.        ,  0.30049848,
         0.01082679],
       [-0.02090571, -0.01053008,  0.03385854,  0.30049848,  1.        ,
         0.01309615],
       [ 0.01468997, -0.00171142,  0.0039523 ,  0.01082679,  0.01309615,
         1.        ]])

In [9]:
fg_n = 200622
fi_n = 151013
hba1c_n = 146806
bmi_n = 806834
t2d_n = 492191
gcn_n = 8421

w_i = [fi_n, fg_n, hba1c_n, t2d_n, bmi_n, gcn_n]

sum_wi = np.sqrt(np.sum(np.square(w_i)))

In [10]:
w = w_i/sum_wi

In [11]:
w

array([0.15270908, 0.20287526, 0.14845483, 0.49771898, 0.81589585,
       0.00851558])

In [12]:
sign_t = np.sign(z)

In [24]:
sign_t = np.nan_to_num(sign_t)
sign_t

array([[ 1., -1., -1.,  1., -1., -1.],
       [-1.,  1.,  1.,  1., -1., -1.],
       [-1., -1.,  1., -1.,  1.,  1.],
       ...,
       [ 0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.]])

In [25]:
w_s = w * sign_t

In [26]:
w_s

array([[ 0.15270908, -0.20287526, -0.14845483,  0.49771898, -0.81589585,
        -0.00851558],
       [-0.15270908,  0.20287526,  0.14845483,  0.49771898, -0.81589585,
        -0.00851558],
       [-0.15270908, -0.20287526,  0.14845483, -0.49771898,  0.81589585,
         0.00851558],
       ...,
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.81589585,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.81589585,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.81589585,
         0.        ]])

In [27]:
w_s[0]

array([ 0.15270908, -0.20287526, -0.14845483,  0.49771898, -0.81589585,
       -0.00851558])

In [28]:
correlation

array([[ 1.        ,  0.19626832,  0.04210314,  0.06468955, -0.02090571,
         0.01468997],
       [ 0.19626832,  1.        ,  0.12033641,  0.11429107, -0.01053008,
        -0.00171142],
       [ 0.04210314,  0.12033641,  1.        ,  0.11435733,  0.03385854,
         0.0039523 ],
       [ 0.06468955,  0.11429107,  0.11435733,  1.        ,  0.30049848,
         0.01082679],
       [-0.02090571, -0.01053008,  0.03385854,  0.30049848,  1.        ,
         0.01309615],
       [ 0.01468997, -0.00171142,  0.0039523 ,  0.01082679,  0.01309615,
         1.        ]])

In [29]:
inv(correlation)

array([[ 1.04370282, -0.19645172, -0.01528876, -0.05419297,  0.03675627,
        -0.01550238],
       [-0.19645172,  1.06465972, -0.10880064, -0.10971955,  0.04368303,
         0.00575379],
       [-0.01528876, -0.10880064,  1.02561883, -0.10219636, -0.00544424,
        -0.00283741],
       [-0.05419297, -0.10971955, -0.10219636,  1.12941239, -0.3381259 ,
        -0.00678753],
       [ 0.03675627,  0.04368303, -0.00544424, -0.3381259 ,  1.10316613,
        -0.01123009],
       [-0.01550238,  0.00575379, -0.00283741, -0.00678753, -0.01123009,
         1.00046935]])

In [30]:
sig = np.linalg.pinv(correlation)

In [31]:
sig

array([[ 1.04370282, -0.19645172, -0.01528876, -0.05419297,  0.03675627,
        -0.01550238],
       [-0.19645172,  1.06465972, -0.10880064, -0.10971955,  0.04368303,
         0.00575379],
       [-0.01528876, -0.10880064,  1.02561883, -0.10219636, -0.00544424,
        -0.00283741],
       [-0.05419297, -0.10971955, -0.10219636,  1.12941239, -0.3381259 ,
        -0.00678753],
       [ 0.03675627,  0.04368303, -0.00544424, -0.3381259 ,  1.10316613,
        -0.01123009],
       [-0.01550238,  0.00575379, -0.00283741, -0.00678753, -0.01123009,
         1.00046935]])

In [32]:
z = np.nan_to_num(z)

In [33]:
z

array([[ 0.39344262, -1.3220339 , -0.12962963,  0.3697479 , -0.19354839,
        -0.22324159],
       [-0.34042553,  1.2195122 ,  1.56666667,  0.38848921, -0.3902439 ,
        -1.8879056 ],
       [-0.09756098, -0.2972973 ,  0.44444444, -1.06923077,  0.7804878 ,
         0.09893993],
       ...,
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.91891892,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.28947368,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.125     ,
         0.        ]])

In [34]:
w = w.reshape(1,6)
w

array([[0.15270908, 0.20287526, 0.14845483, 0.49771898, 0.81589585,
        0.00851558]])

In [35]:
z.shape

(44245766, 6)

In [36]:
sHom_scores = []
count = 1
sig = inv(correlation)
for i in range(z.shape[0]):
    k = np.matmul(np.matmul(w, sig), np.asmatrix(z[i]).T)
    p = np.matmul(np.matmul(w, sig), np.asmatrix(w).T)
    s = (k*k)/p
    score = np.asarray(s).flatten()[0]
    sHom_scores.append(score)
    if count % 1000000 == 0:
        print("-----", count, " iterations -------")
    count+= 1

----- 1000000  iterations -------
----- 2000000  iterations -------
----- 3000000  iterations -------
----- 4000000  iterations -------
----- 5000000  iterations -------
----- 6000000  iterations -------
----- 7000000  iterations -------
----- 8000000  iterations -------
----- 9000000  iterations -------
----- 10000000  iterations -------
----- 11000000  iterations -------
----- 12000000  iterations -------
----- 13000000  iterations -------
----- 14000000  iterations -------
----- 15000000  iterations -------
----- 16000000  iterations -------
----- 17000000  iterations -------
----- 18000000  iterations -------
----- 19000000  iterations -------
----- 20000000  iterations -------
----- 21000000  iterations -------
----- 22000000  iterations -------
----- 23000000  iterations -------
----- 24000000  iterations -------
----- 25000000  iterations -------
----- 26000000  iterations -------
----- 27000000  iterations -------
----- 28000000  iterations -------
----- 29000000  iterations --

In [37]:
np.savetxt('sHom_scores.txt', sHom_scores)

In [8]:
sHom_scores = np.loadtxt('sHom_scores.txt', usecols=range(1))
from scipy.stats import chisquare
from rpy2.robjects import r, pandas2ri

pandas2ri.activate()
p_values = []
count = 0
for s in sHom_scores:
    if(count %1000000 == 0):
        print("------", count, "----------")
    p = np.exp(np.longdouble(r.pchisq(s,1, lower_tail=False, log_p=True)[0]))
    p_values.append(p)
    #6 degree of freedom
    #p_value = stats.chi2.cdf(scores, 6)
    #p_values.append(p_value) 
    count += 1

------ 0 ----------
------ 1000000 ----------
------ 2000000 ----------
------ 3000000 ----------
------ 4000000 ----------
------ 5000000 ----------
------ 6000000 ----------
------ 7000000 ----------
------ 8000000 ----------
------ 9000000 ----------
------ 10000000 ----------
------ 11000000 ----------
------ 12000000 ----------
------ 13000000 ----------
------ 14000000 ----------
------ 15000000 ----------
------ 16000000 ----------
------ 17000000 ----------
------ 18000000 ----------
------ 19000000 ----------
------ 20000000 ----------
------ 21000000 ----------
------ 22000000 ----------
------ 23000000 ----------
------ 24000000 ----------
------ 25000000 ----------
------ 26000000 ----------
------ 27000000 ----------
------ 28000000 ----------
------ 29000000 ----------
------ 30000000 ----------
------ 31000000 ----------
------ 32000000 ----------
------ 33000000 ----------
------ 34000000 ----------
------ 35000000 ----------
------ 36000000 ----------
------ 37000000 -

In [9]:
np.savetxt('sHom_p.txt', p_values)

In [ ]:
p_values

In [38]:
from scipy.stats import chi2
count = 0
sHet_scores = []
cut_off = 0
for i in range(z.shape[0]): 
    T = -1
    true_list, = np.where(np.abs(z[i]) > cut_off)
    w_i = w
    t = z[i]
    cor_i = np.asarray(correlation)
    index = -1
    snp_size = z.shape[1]
    for j in range(z.shape[1]):
        index += 1
        if j not in true_list:
            cor_i = np.delete(cor_i, index, 1)
            cor_i = np.delete(cor_i, index, 0)
            w_i = np.delete(w_i, index)
            snp_size -= 1
            w_i = w_i.reshape(1, snp_size)
            t = np.delete(t, index)
            index -= 1
        else:
            if(t[index] < 0):
                w_i[0][index] = - w_i[0][index]
    sig_i = np.linalg.pinv(cor_i) 
    k = np.matmul(np.matmul(w_i, sig_i),np.asmatrix(t).T)
    p = np.matmul(np.matmul(w_i, sig_i),np.asmatrix(w_i).T)
    s = (k * k)/p
    score = s.flatten().tolist()[0][0]
    if(score > T):
        sHet_scores.append(score)
    else:
        sHet_scores.append(T)
    if count % 1000000 == 0:
        print("-----", count, " iterations -------")
    count+= 1

----- 0  iterations -------


/var/folders/wk/3fc1mkfn61g_f37vy1wk0xfc0000gn/T/ipykernel_2808/260651354.py:29: RuntimeWarning: invalid value encountered in divide
  s = (k * k)/p


----- 1000000  iterations -------
----- 2000000  iterations -------
----- 3000000  iterations -------
----- 4000000  iterations -------
----- 5000000  iterations -------
----- 6000000  iterations -------
----- 7000000  iterations -------
----- 8000000  iterations -------
----- 9000000  iterations -------
----- 10000000  iterations -------
----- 11000000  iterations -------
----- 12000000  iterations -------
----- 13000000  iterations -------
----- 14000000  iterations -------
----- 15000000  iterations -------
----- 16000000  iterations -------
----- 17000000  iterations -------
----- 18000000  iterations -------
----- 19000000  iterations -------
----- 20000000  iterations -------
----- 21000000  iterations -------
----- 22000000  iterations -------
----- 23000000  iterations -------
----- 24000000  iterations -------
----- 25000000  iterations -------
----- 26000000  iterations -------
----- 27000000  iterations -------
----- 28000000  iterations -------
----- 29000000  iterations --

In [39]:
sHet_scores

[0.7514053279405851,
 0.4683617985219453,
 1.8007182872494896,
 0.3052648996498685,
 0.03465283135803519,
 0.4458078646760958,
 2.63108855297614,
 3.7314985842389423,
 0.18602903307935936,
 3.3721057451991867,
 0.39367928646567446,
 0.1964505504799756,
 0.8490851765320969,
 0.895655027759565,
 0.3619585023891917,
 5.727060756414454,
 8.141301340659027,
 5.2985701990115,
 0.24250829790949846,
 3.685665355545387,
 4.853501793977562,
 0.8294374866150331,
 0.17592990038087603,
 9.59351797954059,
 4.124413113107171,
 2.1709525729522774,
 0.015025963207175681,
 5.4864344332519925,
 1.1642262560828038,
 0.015795518254620604,
 0.01399895344576983,
 0.007688239458212806,
 0.34801538957138844,
 2.397072683666996,
 0.00773483633494048,
 1.5352242300442673,
 0.004336900141529629,
 3.3797638715279166,
 0.6301343986420664,
 2.05230285110286,
 0.23180155275356612,
 0.00011241505690349968,
 3.027075377779845,
 0.23238106379026952,
 0.08350696847902211,
 3.998389692414766,
 1.0838218669919042,
 2.20114

In [40]:
np.savetxt('SHet_scores.txt', sHet_scores)

In [10]:
fit_alpha, fit_loc, fit_beta=stats.gamma.fit(sHet_scores)
print(fit_alpha, fit_loc, fit_beta)

NameError: name 'sHet_scores' is not defined

In [30]:
p_zero = stats.gamma.pdf(0, fit_alpha, loc=fit_loc, scale=fit_beta)
SHet_p = []
count = 0
for s in sHet_scores:
    if count % 1000000 == 0:
        print("-----", count, " iterations -------")
    if(s == 0):
        SHet_p.append(p_zero)
    else:
        SHet_p.append(stats.gamma.pdf(s, fit_alpha, loc=fit_loc, scale=fit_beta))
    count+= 1

----- 0  iterations -------
----- 1000000  iterations -------
----- 2000000  iterations -------
----- 3000000  iterations -------
----- 4000000  iterations -------
----- 5000000  iterations -------
----- 6000000  iterations -------
----- 7000000  iterations -------
----- 8000000  iterations -------
----- 9000000  iterations -------
----- 10000000  iterations -------
----- 11000000  iterations -------
----- 12000000  iterations -------
----- 13000000  iterations -------
----- 14000000  iterations -------
----- 15000000  iterations -------
----- 16000000  iterations -------
----- 17000000  iterations -------
----- 18000000  iterations -------
----- 19000000  iterations -------
----- 20000000  iterations -------
----- 21000000  iterations -------
----- 22000000  iterations -------
----- 23000000  iterations -------
----- 24000000  iterations -------
----- 25000000  iterations -------
----- 26000000  iterations -------
----- 27000000  iterations -------
----- 28000000  iterations -------
-

In [31]:
np.savetxt('SHet_p.txt', SHet_p)